# CIFAR - 10 Images 

In [6]:
# Libraries
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, optimizers

# Helper Lib
import numpy as np

In [7]:
# Cifar is a set of 60K images 32 * 32 with 3 color channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)

# Constants
EPOCHS = 20
BATCH_SIZE = 128
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = optimizers.RMSprop()

## One Layer CNN

In [8]:
# Build the model
def build(input_shape, classes):
    model = models.Sequential()
    
    # CONV => RELU => POOL with Dropout
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))
    
    # Flatten => RELU => Drop => Predict
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(classes, activation='softmax'))
    
    return model

In [10]:
# Load the dataset
((X_train, y_train), (X_test, y_test)) = tf.keras.datasets.cifar10.load_data()

In [11]:
# Normalize 
X_train, X_test = X_train / 255. , X_test / 255.

# Cast
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Conver class vectors to binary class vectors
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

In [15]:
model = build(INPUT_SHAPE, CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)

Epoch 1/20
313/313 [==============================] - 37s 119ms/step - loss: 1.7045 - accuracy: 0.3959 - val_loss: 1.5811 - val_accuracy: 0.4620
Epoch 2/20
313/313 [==============================] - 41s 129ms/step - loss: 1.3629 - accuracy: 0.5208 - val_loss: 1.2173 - val_accuracy: 0.5806
Epoch 3/20
313/313 [==============================] - 41s 130ms/step - loss: 1.2361 - accuracy: 0.5649 - val_loss: 1.1457 - val_accuracy: 0.6052
Epoch 4/20
313/313 [==============================] - 41s 132ms/step - loss: 1.1493 - accuracy: 0.5961 - val_loss: 1.1663 - val_accuracy: 0.5959
Epoch 5/20
313/313 [==============================] - 46s 146ms/step - loss: 1.0699 - accuracy: 0.6251 - val_loss: 1.1198 - val_accuracy: 0.6086
Epoch 6/20
313/313 [==============================] - 43s 137ms/step - loss: 1.0138 - accuracy: 0.6468 - val_loss: 1.1085 - val_accuracy: 0.6096
Epoch 7/20
313/313 [==============================] - 44s 141ms/step - loss: 0.9587 - accuracy: 0.6672 - val_loss: 1.0715 - val_ac

In [16]:
# Evaluating the model
score = model.evaluate(X_test, y_test, verbose = VERBOSE)
print('\nTest Score: ', score[0])
print('Test Accuracy: ', score[1])

313/313 [==============================] - 2s 5ms/step - loss: 1.1453 - accuracy: 0.6370

Test Score:  1.1452832221984863
Test Accuracy:  0.6370000243186951


##### Model 1 - One Layer CNN
Accuracy = 63.7%

## Improving the CIFAR-10 performance with a deeper network
- 1st Module: (CONV+CONV+MaxPool+DropOut)
- 2nd Module: (CONV+CONV+MaxPool+DropOut)
- 3rd Module: (CONV+CONV+MaxPool+DropOut)
- 4th Module: (Dense Layer)

In [3]:
def build_model():
    model = models.Sequential()
    
    # 1st Block
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))
    
    # 2nd Block 
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))

    # 3rd Block
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.4))

    # Dense
    model.add(layers.Flatten())
    model.add(layers.Dense(CLASSES, activation='softmax'))
    return model

In [4]:
def load_data():
    
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # Normalize
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    x_train = (x_train - mean) / (std + 1e-7)
    x_test = (x_test - mean) / (std + 1e-7)

    y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

    return x_train, y_train, x_test, y_test

In [9]:
(x_train, y_train, x_test, y_test) = load_data()
deep_model = build_model()
deep_model.compile(loss='categorical_crossentropy',
                   optimizer='RMSprop',
                   metrics=['accuracy'])

# Train
batch_size = 64
deep_model.fit(x_train, y_train, batch_size=batch_size,
               epochs=EPOCHS, validation_data=(x_test, y_test))

Epoch 1/20
 90/782 [==>...........................] - ETA: 5:01 - loss: 2.7943 - accuracy: 0.2844

KeyboardInterrupt: 